# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [25]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_id_text = []
    for sentence in source_text.split('\n'):
        source_id_list = []
        for word in sentence.split():
            source_id_list.append(source_vocab_to_int[word])
        source_id_text.append(source_id_list)
        
    target_id_text = []
    for sentence in target_text.split('\n'):
        target_id_list = []
        for word in sentence.split():
            target_id_list.append(target_vocab_to_int[word])
        target_id_list.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(target_id_list)
    
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/arturops/anaconda3/envs/rnn/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [29]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_             = tf.placeholder(tf.int32,   shape=[None,None], name='input')
    targets            = tf.placeholder(tf.int32,   shape=[None,None], name='targets')
    learning_rate      = tf.placeholder(tf.float32, shape=None,        name='learning_rate')
    keep_prob          = tf.placeholder(tf.float32, shape=None,        name='keep_prob')
    target_seq_len     = tf.placeholder(tf.int32,   shape=[None],      name='target_sequence_length')
    max_target_seq_len = tf.reduce_max(input_tensor=target_seq_len,    name='max_target_len')
    source_seq_len     = tf.placeholder(tf.int32,   shape=[None],      name='source_sequence_length')
    return input_, targets, learning_rate, keep_prob, target_seq_len, max_target_seq_len, source_seq_len


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [37]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    # remove the last column on the batch (last sequence), strides=[1,1] means one colum
    target_data_removed_end = tf.strided_slice(target_data, begin=[0,0], end=[batch_size,-1], strides=[1,1])
    # append the <GO> int ID to the input data of the decoder
    #   a. Create a tensor with dimensions similar to the batch. Then, GO ID can be appended to all rows of the sequence
    GO_tensor_to_concat = tf.fill(dims=[batch_size,1], value=target_vocab_to_int['<GO>'])
    #   b. Append the GO tensor to the whole batch using axis=1 because appends in column shape
    preprocess_target_data = tf.concat([GO_tensor_to_concat,target_data_removed_end],axis=1)
            
    
    return preprocess_target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [88]:
from imp import reload
reload(tests)

def createLSTMcell(rnn_size, keep_prob=None):
    """
    Create LSTM cell with give Dropout probability 1-keep_prob
    :param rnn_size: number of LSTM cells
    :keep_prob: Dropout keep probability
    """
    lstm = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    drop_lstm = tf.contrib.rnn.DropoutWrapper(lstm,input_keep_prob=keep_prob)#,output_keep_prob=keep_prob,state_keep_prob=keep_prob)
    
    return drop_lstm
    
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    embed = tf.contrib.layers.embed_sequence(ids=rnn_inputs, vocab_size=source_vocab_size, embed_dim=encoding_embedding_size)
   
    multi_lstm = tf.contrib.rnn.MultiRNNCell([createLSTMcell(rnn_size,keep_prob) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(multi_lstm, embed, sequence_length=source_sequence_length, dtype= tf.float32)
    
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [111]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    # :param Time_major: makes computation faster, returns Tensor in time major mode rather than Batch
    # This code fails if set to time_major set to True, do not set it
    #with tf.variable_scope("decoder"):
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
        
        
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper,\
                                                        initial_state=encoder_state, output_layer=output_layer)
        
    BasicDecoderOutput,_ = tf.contrib.seq2seq.dynamic_decode(basic_decoder, impute_finished=True,\
                                                             maximum_iterations=max_summary_length)
        
    return BasicDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [112]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    #with tf.variable_scope("decoder", reuse=True):
    
    #column vector of ID's used in each sentence feed on beginning of first sequence of each batch
    start_id = tf.constant([start_of_sequence_id], dtype=tf.int32)
    start_id_tensor = tf.tile(start_id, [batch_size],name='start_tokens')
        
    # GreedyEmbeddingHelper - start_tokens = 1D Tensor of START IDs, end_token = scalar value of END ID
    dec_helper_greedy = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_id_tensor, \
                                                                     end_of_sequence_id)
        
    dec_infer_basic = tf.contrib.seq2seq.BasicDecoder(dec_cell, dec_helper_greedy, encoder_state, output_layer=output_layer)
        
    dec_infer_output,_ = tf.contrib.seq2seq.dynamic_decode(dec_infer_basic, impute_finished=True,\
                                                           maximum_iterations=max_target_sequence_length)
        
    
    return dec_infer_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [121]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # 1. EMBEDDING
    # inputs had been preprocessed in decoder_inputs, now we just need to do the embed look_up
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]), name='decoder_embed')
    # dec_input - id's of the words to decode
    dec_embedding_input = tf.nn.embedding_lookup(dec_embeddings, dec_input,name='decoder_embed_look_up')
    
    # 2. BUILD DECODER
    decoder_stack_lstm = tf.contrib.rnn.MultiRNNCell([createLSTMcell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    # 3. OUTPUT LAYER that maps output of decoder to elements of vocabulary
    
    output_layer = Dense(target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1), \
                         name='output_layer')
    
    
    #--------------------------------------------------------------------------------------------------------------
    # The dec_embeddings MUST BE PASSED TO dec_inference, while the EMBEDDED_LOOKUP result is PASSED to dec train
    #--------------------------------------------------------------------------------------------------------------
    with tf.variable_scope("decoder"):
        train_logits = decoding_layer_train(encoder_state, decoder_stack_lstm, dec_embedding_input,\
                                            target_sequence_length, max_target_sequence_length, \
                                            output_layer, keep_prob)
        
    with tf.variable_scope("decoder", reuse=True):
        # dropout for inference ASSUMED = 1 since it is not training and all waits have been set in dec_train
        # so the Dropout should be at training time
        keep_prob_infer = tf.constant(1.0,dtype=tf.float32,name='dec_infer_Dropout')
        inference_logits = decoding_layer_infer(encoder_state, decoder_stack_lstm, dec_embeddings, \
                                                target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], \
                                                max_target_sequence_length, target_vocab_size, output_layer,\
                                                batch_size, keep_prob_infer)
    
    return train_logits, inference_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [124]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param max_target_sentence_length: Length of the longest target sentence
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # 1. Apply embedding to the input data for the encoder (embedded done inside encoding_layer())
    # 2. Create the encoding_layer
    BasicEncoderOuput, encoder_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,\
                                       source_sequence_length, source_vocab_size, enc_embedding_size)
    
    # 3. Preprocess the target data that will be feed to the decoder
    preprocess_dec_data = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # 4. Apply embedding to the input target data for the decoder (embedded done inside decoding_layer())
    # 5. Create the decoding_layer
    train_BasicDecoderOutput, inference_BasicDecoderOutput = decoding_layer(preprocess_dec_data, encoder_state,\
                                                                            target_sequence_length,\
                                                                            max_target_sentence_length, rnn_size,\
                                                                            num_layers, target_vocab_to_int,\
                                                                            target_vocab_size, batch_size,\
                                                                            keep_prob, dec_embedding_size)
    
    return train_BasicDecoderOutput, inference_BasicDecoderOutput


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [135]:
# Number of Epochs
epochs = 5
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 1
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 5

### Build the Graph
Build the graph using the neural network you implemented.

In [136]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [137]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [138]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/2154 - Train Accuracy: 0.2105, Validation Accuracy: 0.2841, Loss: 5.0353
Epoch   0 Batch   10/2154 - Train Accuracy: 0.2117, Validation Accuracy: 0.2841, Loss: 4.4711
Epoch   0 Batch   15/2154 - Train Accuracy: 0.2547, Validation Accuracy: 0.3146, Loss: 4.0545
Epoch   0 Batch   20/2154 - Train Accuracy: 0.2836, Validation Accuracy: 0.3210, Loss: 3.6104
Epoch   0 Batch   25/2154 - Train Accuracy: 0.2992, Validation Accuracy: 0.3494, Loss: 3.4997
Epoch   0 Batch   30/2154 - Train Accuracy: 0.2773, Validation Accuracy: 0.3409, Loss: 3.3205
Epoch   0 Batch   35/2154 - Train Accuracy: 0.3180, Validation Accuracy: 0.3516, Loss: 3.1711
Epoch   0 Batch   40/2154 - Train Accuracy: 0.2722, Validation Accuracy: 0.3530, Loss: 3.1970
Epoch   0 Batch   45/2154 - Train Accuracy: 0.2903, Validation Accuracy: 0.3672, Loss: 3.1638
Epoch   0 Batch   50/2154 - Train Accuracy: 0.2796, Validation Accuracy: 0.3750, Loss: 3.0745
Epoch   0 Batch   55/2154 - Train Accuracy: 0.3383, Validati

Epoch   0 Batch  445/2154 - Train Accuracy: 0.4570, Validation Accuracy: 0.5291, Loss: 0.9428
Epoch   0 Batch  450/2154 - Train Accuracy: 0.5625, Validation Accuracy: 0.5312, Loss: 0.8350
Epoch   0 Batch  455/2154 - Train Accuracy: 0.4852, Validation Accuracy: 0.5298, Loss: 0.9028
Epoch   0 Batch  460/2154 - Train Accuracy: 0.5602, Validation Accuracy: 0.5256, Loss: 0.8460
Epoch   0 Batch  465/2154 - Train Accuracy: 0.5000, Validation Accuracy: 0.5263, Loss: 0.8759
Epoch   0 Batch  470/2154 - Train Accuracy: 0.4696, Validation Accuracy: 0.5291, Loss: 0.9512
Epoch   0 Batch  475/2154 - Train Accuracy: 0.4910, Validation Accuracy: 0.5391, Loss: 0.8317
Epoch   0 Batch  480/2154 - Train Accuracy: 0.5461, Validation Accuracy: 0.5241, Loss: 0.7718
Epoch   0 Batch  485/2154 - Train Accuracy: 0.5258, Validation Accuracy: 0.5263, Loss: 0.7635
Epoch   0 Batch  490/2154 - Train Accuracy: 0.5500, Validation Accuracy: 0.5284, Loss: 0.8262
Epoch   0 Batch  495/2154 - Train Accuracy: 0.5900, Validati

Epoch   0 Batch  885/2154 - Train Accuracy: 0.6704, Validation Accuracy: 0.6420, Loss: 0.5785
Epoch   0 Batch  890/2154 - Train Accuracy: 0.6579, Validation Accuracy: 0.6413, Loss: 0.6096
Epoch   0 Batch  895/2154 - Train Accuracy: 0.5831, Validation Accuracy: 0.6179, Loss: 0.6275
Epoch   0 Batch  900/2154 - Train Accuracy: 0.6234, Validation Accuracy: 0.6044, Loss: 0.6402
Epoch   0 Batch  905/2154 - Train Accuracy: 0.6672, Validation Accuracy: 0.6264, Loss: 0.5646
Epoch   0 Batch  910/2154 - Train Accuracy: 0.6003, Validation Accuracy: 0.6243, Loss: 0.5988
Epoch   0 Batch  915/2154 - Train Accuracy: 0.6853, Validation Accuracy: 0.6222, Loss: 0.5340
Epoch   0 Batch  920/2154 - Train Accuracy: 0.6503, Validation Accuracy: 0.6328, Loss: 0.5466
Epoch   0 Batch  925/2154 - Train Accuracy: 0.6234, Validation Accuracy: 0.6278, Loss: 0.5497
Epoch   0 Batch  930/2154 - Train Accuracy: 0.6969, Validation Accuracy: 0.6520, Loss: 0.5719
Epoch   0 Batch  935/2154 - Train Accuracy: 0.7212, Validati

Epoch   0 Batch 1325/2154 - Train Accuracy: 0.7240, Validation Accuracy: 0.7138, Loss: 0.3946
Epoch   0 Batch 1330/2154 - Train Accuracy: 0.7945, Validation Accuracy: 0.7244, Loss: 0.3977
Epoch   0 Batch 1335/2154 - Train Accuracy: 0.7179, Validation Accuracy: 0.7294, Loss: 0.4682
Epoch   0 Batch 1340/2154 - Train Accuracy: 0.7477, Validation Accuracy: 0.7486, Loss: 0.4219
Epoch   0 Batch 1345/2154 - Train Accuracy: 0.7212, Validation Accuracy: 0.7301, Loss: 0.4572
Epoch   0 Batch 1350/2154 - Train Accuracy: 0.7320, Validation Accuracy: 0.7280, Loss: 0.4202
Epoch   0 Batch 1355/2154 - Train Accuracy: 0.7266, Validation Accuracy: 0.7457, Loss: 0.4497
Epoch   0 Batch 1360/2154 - Train Accuracy: 0.7253, Validation Accuracy: 0.7266, Loss: 0.4216
Epoch   0 Batch 1365/2154 - Train Accuracy: 0.7336, Validation Accuracy: 0.7372, Loss: 0.4089
Epoch   0 Batch 1370/2154 - Train Accuracy: 0.7297, Validation Accuracy: 0.7386, Loss: 0.4133
Epoch   0 Batch 1375/2154 - Train Accuracy: 0.7352, Validati

Epoch   0 Batch 1765/2154 - Train Accuracy: 0.8257, Validation Accuracy: 0.8040, Loss: 0.3526
Epoch   0 Batch 1770/2154 - Train Accuracy: 0.8008, Validation Accuracy: 0.7997, Loss: 0.3091
Epoch   0 Batch 1775/2154 - Train Accuracy: 0.8016, Validation Accuracy: 0.8097, Loss: 0.3130
Epoch   0 Batch 1780/2154 - Train Accuracy: 0.8359, Validation Accuracy: 0.8068, Loss: 0.3035
Epoch   0 Batch 1785/2154 - Train Accuracy: 0.8207, Validation Accuracy: 0.7898, Loss: 0.3069
Epoch   0 Batch 1790/2154 - Train Accuracy: 0.8073, Validation Accuracy: 0.8075, Loss: 0.3044
Epoch   0 Batch 1795/2154 - Train Accuracy: 0.8000, Validation Accuracy: 0.8054, Loss: 0.2945
Epoch   0 Batch 1800/2154 - Train Accuracy: 0.8187, Validation Accuracy: 0.8161, Loss: 0.2958
Epoch   0 Batch 1805/2154 - Train Accuracy: 0.7734, Validation Accuracy: 0.8068, Loss: 0.3289
Epoch   0 Batch 1810/2154 - Train Accuracy: 0.7523, Validation Accuracy: 0.7969, Loss: 0.3209
Epoch   0 Batch 1815/2154 - Train Accuracy: 0.8094, Validati

...

Epoch   4 Batch  205/2154 - Train Accuracy: 0.9867, Validation Accuracy: 0.9553, Loss: 0.0208
Epoch   4 Batch  210/2154 - Train Accuracy: 0.9836, Validation Accuracy: 0.9474, Loss: 0.0243
Epoch   4 Batch  215/2154 - Train Accuracy: 0.9241, Validation Accuracy: 0.9368, Loss: 0.0355
Epoch   4 Batch  220/2154 - Train Accuracy: 0.9617, Validation Accuracy: 0.9624, Loss: 0.0245
Epoch   4 Batch  225/2154 - Train Accuracy: 0.9641, Validation Accuracy: 0.9354, Loss: 0.0205
Epoch   4 Batch  230/2154 - Train Accuracy: 0.9695, Validation Accuracy: 0.9375, Loss: 0.0305
Epoch   4 Batch  235/2154 - Train Accuracy: 0.9773, Validation Accuracy: 0.9425, Loss: 0.0235
Epoch   4 Batch  240/2154 - Train Accuracy: 0.9469, Validation Accuracy: 0.9439, Loss: 0.0315
Epoch   4 Batch  245/2154 - Train Accuracy: 0.9630, Validation Accuracy: 0.9467, Loss: 0.0351
Epoch   4 Batch  250/2154 - Train Accuracy: 0.9586, Validation Accuracy: 0.9496, Loss: 0.0327
Epoch   4 Batch  255/2154 - Train Accuracy: 0.9609, Validati

Epoch   4 Batch  645/2154 - Train Accuracy: 0.9561, Validation Accuracy: 0.9411, Loss: 0.0290
Epoch   4 Batch  650/2154 - Train Accuracy: 0.9164, Validation Accuracy: 0.9560, Loss: 0.0335
Epoch   4 Batch  655/2154 - Train Accuracy: 0.9556, Validation Accuracy: 0.9432, Loss: 0.0395
Epoch   4 Batch  660/2154 - Train Accuracy: 0.9344, Validation Accuracy: 0.9446, Loss: 0.0354
Epoch   4 Batch  665/2154 - Train Accuracy: 0.9570, Validation Accuracy: 0.9574, Loss: 0.0223
Epoch   4 Batch  670/2154 - Train Accuracy: 0.9836, Validation Accuracy: 0.9510, Loss: 0.0238
Epoch   4 Batch  675/2154 - Train Accuracy: 0.9695, Validation Accuracy: 0.9375, Loss: 0.0326
Epoch   4 Batch  680/2154 - Train Accuracy: 0.9630, Validation Accuracy: 0.9411, Loss: 0.0253
Epoch   4 Batch  685/2154 - Train Accuracy: 0.9703, Validation Accuracy: 0.9517, Loss: 0.0233
Epoch   4 Batch  690/2154 - Train Accuracy: 0.9273, Validation Accuracy: 0.9524, Loss: 0.0446
Epoch   4 Batch  695/2154 - Train Accuracy: 0.9938, Validati

Epoch   4 Batch 1085/2154 - Train Accuracy: 0.9836, Validation Accuracy: 0.9659, Loss: 0.0183
Epoch   4 Batch 1090/2154 - Train Accuracy: 0.9622, Validation Accuracy: 0.9588, Loss: 0.0223
Epoch   4 Batch 1095/2154 - Train Accuracy: 0.9969, Validation Accuracy: 0.9588, Loss: 0.0258
Epoch   4 Batch 1100/2154 - Train Accuracy: 0.9516, Validation Accuracy: 0.9560, Loss: 0.0373
Epoch   4 Batch 1105/2154 - Train Accuracy: 0.9664, Validation Accuracy: 0.9361, Loss: 0.0290
Epoch   4 Batch 1110/2154 - Train Accuracy: 0.9609, Validation Accuracy: 0.9695, Loss: 0.0318
Epoch   4 Batch 1115/2154 - Train Accuracy: 0.9474, Validation Accuracy: 0.9688, Loss: 0.0259
Epoch   4 Batch 1120/2154 - Train Accuracy: 0.9516, Validation Accuracy: 0.9688, Loss: 0.0247
Epoch   4 Batch 1125/2154 - Train Accuracy: 0.9416, Validation Accuracy: 0.9688, Loss: 0.0190
Epoch   4 Batch 1130/2154 - Train Accuracy: 0.9646, Validation Accuracy: 0.9560, Loss: 0.0286
Epoch   4 Batch 1135/2154 - Train Accuracy: 0.9539, Validati

Epoch   4 Batch 1525/2154 - Train Accuracy: 0.9672, Validation Accuracy: 0.9702, Loss: 0.0240
Epoch   4 Batch 1530/2154 - Train Accuracy: 0.9646, Validation Accuracy: 0.9688, Loss: 0.0335
Epoch   4 Batch 1535/2154 - Train Accuracy: 0.9945, Validation Accuracy: 0.9482, Loss: 0.0219
Epoch   4 Batch 1540/2154 - Train Accuracy: 0.9617, Validation Accuracy: 0.9595, Loss: 0.0358
Epoch   4 Batch 1545/2154 - Train Accuracy: 0.9435, Validation Accuracy: 0.9688, Loss: 0.0330
Epoch   4 Batch 1550/2154 - Train Accuracy: 0.9641, Validation Accuracy: 0.9688, Loss: 0.0288
Epoch   4 Batch 1555/2154 - Train Accuracy: 0.9688, Validation Accuracy: 0.9688, Loss: 0.0277
Epoch   4 Batch 1560/2154 - Train Accuracy: 0.9367, Validation Accuracy: 0.9893, Loss: 0.0277
Epoch   4 Batch 1565/2154 - Train Accuracy: 0.9531, Validation Accuracy: 0.9787, Loss: 0.0302
Epoch   4 Batch 1570/2154 - Train Accuracy: 0.9578, Validation Accuracy: 0.9787, Loss: 0.0201
Epoch   4 Batch 1575/2154 - Train Accuracy: 0.9844, Validati

Epoch   4 Batch 1965/2154 - Train Accuracy: 0.9479, Validation Accuracy: 0.9624, Loss: 0.0265
Epoch   4 Batch 1970/2154 - Train Accuracy: 0.9539, Validation Accuracy: 0.9496, Loss: 0.0315
Epoch   4 Batch 1975/2154 - Train Accuracy: 0.9297, Validation Accuracy: 0.9545, Loss: 0.0256
Epoch   4 Batch 1980/2154 - Train Accuracy: 0.9461, Validation Accuracy: 0.9716, Loss: 0.0295
Epoch   4 Batch 1985/2154 - Train Accuracy: 0.9671, Validation Accuracy: 0.9673, Loss: 0.0336
Epoch   4 Batch 1990/2154 - Train Accuracy: 0.9622, Validation Accuracy: 0.9616, Loss: 0.0170
Epoch   4 Batch 1995/2154 - Train Accuracy: 0.9597, Validation Accuracy: 0.9716, Loss: 0.0395
Epoch   4 Batch 2000/2154 - Train Accuracy: 0.9523, Validation Accuracy: 0.9730, Loss: 0.0412
Epoch   4 Batch 2005/2154 - Train Accuracy: 0.9605, Validation Accuracy: 0.9609, Loss: 0.0180
Epoch   4 Batch 2010/2154 - Train Accuracy: 0.9433, Validation Accuracy: 0.9553, Loss: 0.0432
Epoch   4 Batch 2015/2154 - Train Accuracy: 0.9650, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [139]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [140]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [141]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    sentence_lowercase = sentence.lower() 
    words2int = []
    for word in sentence.split():
        if word in vocab_to_int:
            words2int.append(vocab_to_int[word])
        else:
            words2int.append(vocab_to_int['<UNK>'])
    
    return words2int


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [146]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [147, 31, 115, 154, 82, 8, 193]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [205, 316, 142, 296, 164, 140, 247, 344, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.